In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
from optuna.integration import LightGBMPruningCallback
import sqlite3
import ray
from autogluon.tabular import TabularPredictor

/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-26 20:04:28,871	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
# Reunir los DataFrames df_full y df_pendientes por PRODUCT_ID, CUSTOMER_ID y PERIODO, agregar las 
# columnas de df_pendientes a df_full
df_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))

In [3]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS','MES_PROBLEMATICO']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Hacer que A_PREDECIR sea boolean si es 'S' vale True, si es 'N' False
df_full['A_PREDECIR'] = df_full['A_PREDECIR'].map({'S': True, 'N': False})


In [4]:
# Filtrar filas donde el target es válido (ni NaN ni Inf)
df_full = df_full[df_full['CLASE_DELTA'].notnull() & np.isfinite(df_full['CLASE_DELTA'])]

In [5]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA' si existen
cols_to_drop = [col for col in ['CLASE', 'CLASE_DELTA'] if col in df_full.columns]
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=cols_to_drop)
# Filtrar en y que el periodo sea menor o igual a 201910 y que la columna exista
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()

30

In [6]:
# Definir los periodos de validación 201910
#periodos_valid = [201910]
periodos_valid = [201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
del X, y
gc.collect()

0

In [7]:
from autogluon.tabular import TabularPredictor

#ray.init(include_dashboard=False)

# Dataset de entrenamiento (PERIODO < 201910)
df_train = X_train.copy()
df_train['CLASE_DELTA'] = y_train

# Dataset de validación (PERIODO == 201910)
df_val = X_val_list[0].copy()
df_val['CLASE_DELTA'] = y_val_list[0]

# Entrenamiento con AutoGluon
predictor = TabularPredictor(
    label='CLASE_DELTA',
    problem_type='regression'
).fit(
    train_data=df_train,
    tuning_data=df_val,
    time_limit=900,
    presets='best_quality',
    use_bag_holdout=True  # <--- esta línea resuelve tu error
)


No path specified. Models will be saved in: "AutogluonModels/ag-20250626_230517"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.23
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Apr 17 19:20:47 UTC 2
CPU Count:          28
Memory Avail:       103.08 GB / 125.58 GB (82.1%)
Disk Space Avail:   419.32 GB / 543.17 GB (77.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to False. Reason: Skip dynamic_stacking when use_bag_holdout is enabled. (use_bag_holdout=True)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "/home/pablo/Documentos/labo3-2025v/AutogluonModels/ag-20250626_230517"
Train Data Rows:    15084267
Train Data Columns: 95
Tuning Data Rows:    544570
Tuning Data Columns: 95
Labe

2025-06-26 20:20:27,925	ERROR worker.py:409 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): The worker died unexpectedly while executing this task. Check python-core-worker-*.log files for more information.


In [8]:
# Cargo los datos sobre los que quiero hacer predicciones
gc.collect()
df_pred_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
df_pred_full = df_pred_full[df_pred_full['PERIODO'] == 201910].drop(columns=['CLASE', 'CLASE_DELTA'])
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
df_pendientes = df_pendientes[df_pendientes['PERIODO'] == 201910]
df_pred_full = df_pred_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))


In [9]:
# Filtrar solo los que tengan la columna A_PREDECIR con valor 1
df_pred_full = df_pred_full[df_pred_full['A_PREDECIR'] == 'S']
# Hacer que A_PREDECIR sea boolean si es 'S' vale True, si es 'N' False
df_pred_full['A_PREDECIR'] = df_pred_full['A_PREDECIR'].map({'S': True, 'N': False})

# Agregar a df_pred_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_pred_full['MES_PROBLEMATICO'] = df_pred_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')

In [10]:
# Eliminar del dataframe df_pred_full la columna 'PREDICCIONES'
if 'PREDICCIONES' in df_pred_full.columns:
    df_pred_full.drop(columns=['PREDICCIONES'], inplace=True)

In [11]:
if not df_pred_full.empty:
    # Eliminar la columna target si está presente (puede venir con NaN)
    df_pred_full = df_pred_full.drop(columns=["CLASE_DELTA"], errors="ignore")
    
    # Hacer predicciones
    predictions = predictor.predict(df_pred_full)
    
    # Guardar las predicciones en el DataFrame
    df_pred_full["PREDICCIONES"] = predictions
else:
    print("df_pred_full está vacío, no se generaron predicciones.")
